In [83]:
label_to_coordinates = {
    "1-1": (0, 0), "1-2": (0.6, 0), "1-3": (1.2, 0), "1-4": (1.8, 0), "1-5": (2.4, 0), "1-6": (3.0, 0),"1-7": (3.6, 0), "1-8": (4.2, 0), "1-9": (4.8, 0), "1-10": (5.4, 0), "1-11": (6.0, 0),
    "2-1": (0, 0.6), "2-11": (6.0, 0.6),
    "3-1": (0, 1.2), "3-11": (6.0, 1.2),
    "4-1": (0, 1.8), "4-11": (6.0, 1.8),
    "5-1": (0, 2.4), "5-11": (6.0, 2.4),
    "6-1": (0, 3.0), "6-2": (0.6, 3.0), "6-3": (1.2, 3.0), "6-4": (1.8, 3.0), "6-5": (2.4, 3.0),"6-6": (3.0, 3.0), "6-7": (3.6, 3.0), "6-8": (4.2, 3.0), "6-9": (4.8, 3.0), "6-10": (5.4, 3.0), "6-11": (6.0, 3.0),
    "7-1": (0, 3.6), "7-11": (6.0, 3.6),
    "8-1": (0, 4.2), "8-11": (6.0, 4.2),
    "9-1": (0, 4.8), "9-11": (6.0, 4.8),
    "10-1": (0, 5.4), "10-11": (6.0, 5.4),
    "11-1": (0, 6.0), "11-2": (0.6, 6.0), "11-3": (1.2, 6.0), "11-4": (1.8, 6.0), "11-5": (2.4, 6.0),"11-6": (3.0, 6.0), "11-7": (3.6, 6.0), "11-8": (4.2, 6.0), "11-9": (4.8, 6.0), "11-10": (5.4, 6.0), "11-11": (6.0, 6.0)
}
label_mapping = {
    '11': '1-1','10': '1-2','9': '1-3','8': '1-4','7': '1-5','6': '1-6','5': '1-7','4': '1-8','3': '1-9','2': '1-10','1': '1-11',
    '12': '2-1','30': '2-11',
    '13': '3-1','29': '3-11',
    '14': '4-1','28': '4-11',
    '15': '5-1','27': '5-11',
    '16': '6-1','17': '6-2','18': '6-3','19': '6-4','20': '6-5','21': '6-6','22': '6-7','23': '6-8','24': '6-9','25': '6-10','26': '6-11',
    '49': '7-1','31': '7-11',
    '48': '8-1','32': '8-11',
    '47': '9-1','33': '9-11',
    '46': '10-1','34': '10-11',
    '45': '11-1','44': '11-2','43': '11-3','42': '11-4','41': '11-5','40': '11-6','39': '11-7','38': '11-8','37': '11-9','36': '11-10','35': '11-11'
}

In [84]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from tensorflow import keras
import tensorflow as tf
import joblib  # 用於保存模型
import json
from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedShuffleSplit

In [85]:
selected_columns = ['Label',
                        'AP1_Distance (mm)','AP4_Distance (mm)',
                        'AP1_StdDev (mm)','AP4_StdDev (mm)',
                        'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

X_testing_selected_columns = [
                        'AP1_Distance (mm)','AP4_Distance (mm)',
                        'AP1_StdDev (mm)','AP4_StdDev (mm)',
                        'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

label_column = 'Label'
target_column = 'Label'


#  'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi'
# 'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
# 'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',

In [86]:
ap = 'AP1&AP4'
root = 'AP1 AP4 best (MDE_0.0093Accuracy_0.9919 )'

dataamount = 20
N_val = 4

N_train = dataamount # 訓練集每個類別至少要有 N_train 筆資料
test_val_ratio = 1  # 剩餘資料中，50% 作為驗證集，50% 作為測試集

weekrepresent = [['0','0'],['0','1'],['1','2'],['2','3'],['3','4'],['4','5']]
alldatadate = ['2024_12_21','2024_12_27','2025_01_03','2025_01_10','2025_02_28']
number_of_week = len(alldatadate)

ALL_trainingtime = []
ALL_loss = []
ALL_accuracy = []
ALL_mean_mde = []

In [87]:
# 載入 regressor
base_model_reg = joblib.load(f'{root}/regressor_model_{ap}_best_{weekrepresent[0][0]}week_to_{weekrepresent[0][1]}week_{dataamount}dataPerRP.pkl')
# 載入 regressor dnn
base_model_dnn = load_model(f'{root}/regressor_dnn_{ap}_best_{weekrepresent[0][0]}week_to_{weekrepresent[0][1]}week_{dataamount}dataPerRP.h5')
# 載入 scaler
scaler = joblib.load(f'{root}/scaler_regressor_dnn_{ap}_best_0.pkl')

# 讀取測試資料 2024_12_21   2024_12_27   2025_01_03   2025_01_10   2025_02_28
test_file_path = f"timestamp_allignment_Balanced_{alldatadate[0]}_rtt_logs.csv"  # 測試資料的檔案名稱
date_test = f"{alldatadate[0]}"
modelname = f"regressor_DNN {ap}s BEST_{dataamount}data_{alldatadate[0]}"
data = pd.read_csv(test_file_path, usecols=selected_columns)

# DNN transfer learnging 凍結所有層
for layer in base_model_dnn.layers[:-1]:  # 除了最後一層 (Output Layer)
    layer.trainable = False

# 確認哪些層可訓練
base_model_dnn.summary()

base_model_dnn.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])


data_imputed = data.groupby(label_column).apply(
    lambda group: group.fillna(group.mean())
).reset_index()

data_imputed


reverse_label_mapping = {v: int(k) - 1 for k, v in label_mapping.items()}  # 讓數字標籤 -1

y = data_imputed[target_column]

y_numeric = y.map(reverse_label_mapping)
y_numeric

# 把label部分拿掉
X = data_imputed.drop(columns=['level_1','Label'])
print(X.head())

# 轉為 DataFrame 方便操作
data = pd.DataFrame(X)
data['label'] = y_numeric  # 加入 label 欄位

train_data_full = data.groupby('label', group_keys=False).sample(n=N_train, replace=False, random_state=42)
train_data_full


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 49)             │         3,185 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,467 (79.95 KB)

 Trainable params: 3,185 (12.44 KB)

 Non-trainable params: 17,280 (67.50 KB)

 Optimizer params: 2 (12.00 B)

   AP1_Distance (mm)  AP4_Distance (mm)  AP1_Rssi  AP2_Rssi  AP3_Rssi  \
0             4077.0            13164.0     -63.0     -74.0     -52.0   
1             4038.0            12876.0     -63.0     -75.0     -47.0   
2             3849.0            13608.0     -55.0     -75.0     -51.0   
3             4228.0            13135.0     -58.0     -75.0     -51.0   
4             4155.0            12871.0     -63.0     -74.0     -46.0   

   AP4_Rssi  AP1_StdDev (mm)  AP4_StdDev (mm)  
0     -68.0           1153.0           1116.0  
1     -64.0           1374.0           1219.0  
2     -65.0           1423.0           1023.0  
3     -69.0           1426.0           1619.0  
4     -67.0            967.0           1081.0  


,AP1_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP4_StdDev (mm),label
1038,444.0,7366.0,-60.0,-71.0,-64.0,-52.0,791.0,381.0,0
1020,444.0,7444.0,-59.0,-71.0,-62.0,-49.0,905.0,295.0,0
1125,327.0,7353.0,-59.0,-71.0,-63.0,-50.0,757.0,792.0,0
1102,483.0,6987.0,-59.0,-71.0,-62.0,-50.0,743.0,1011.0,0
826,483.0,7353.0,-60.0,-71.0,-56.0,-54.0,729.0,410.0,0
...,...,...,...,...,...,...,...,...,...
16769,6313.0,7666.0,-71.0,-74.0,-60.0,-61.0,946.0,756.0,48
16647,5903.0,7666.0,-70.0,-76.0,-58.0,-60.0,1071.0,731.0,48
16719,6343.0,7822.0,-71.0,-75.0,-59.0,-60.0,892.0,707.0,48
16567,6313.0,7588.0,-72.0,-74.0,-61.0,-61.0,929.0,751.0,48


In [88]:
if N_val > 0:
    sss = StratifiedShuffleSplit(n_splits=1, test_size=N_val / N_train) # , random_state=42
    train_index, val_index = next(sss.split(train_data_full.drop(columns=['label']), train_data_full['label']))
    train_data = train_data_full.iloc[train_index]
    val_data = train_data_full.iloc[val_index]
    
else:
    val_data = pd.DataFrame(columns=data.columns)  # 若沒有 validation data，建立空 DataFrame\
    train_data = train_data_full

# 剩下的資料（未被抽入 train_data_full 的部分）直接作為 test set
remaining_data = data.drop(train_data_full.index)
train_data



,AP1_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP4_StdDev (mm),label
2880,1948.000000,8799.0,-61.00000,-81.000000,-53.0,-64.0,1077.000000,1076.0,5
1239,3335.000000,8657.0,-58.00000,-70.000000,-49.0,-60.0,394.000000,2299.0,9
11584,4687.371025,8174.0,-60.95053,-70.165493,-50.0,-57.0,828.964664,764.0,14
3591,219.000000,8330.0,-60.00000,-69.000000,-54.0,-60.0,319.000000,1016.0,3
4638,2632.000000,2412.0,-61.00000,-71.000000,-54.0,-55.0,353.000000,477.0,33
...,...,...,...,...,...,...,...,...,...
1143,444.000000,7275.0,-59.00000,-71.000000,-67.0,-51.0,724.000000,1564.0,0
15964,-219.000000,6133.0,-61.00000,-71.000000,-60.0,-58.0,1163.000000,2383.0,22
2829,1772.000000,8877.0,-61.00000,-81.000000,-53.0,-64.0,1146.000000,994.0,5
5682,3452.000000,1914.0,-53.00000,-66.000000,-64.0,-52.0,331.000000,418.0,35


In [89]:
# 訓練 regressor 並儲存
ap1_data = train_data[['AP1_Rssi', 'AP1_Distance (mm)']].dropna().rename(
    columns={'AP1_Rssi': 'Rssi', 'AP1_Distance (mm)': 'Distance'}
)
ap4_data = train_data[['AP4_Rssi', 'AP4_Distance (mm)']].dropna().rename(
    columns={'AP4_Rssi': 'Rssi', 'AP4_Distance (mm)': 'Distance'}
)

train_data_reg = pd.concat([ap1_data, ap4_data], ignore_index=True)


X_train_reg = train_data_reg[['Rssi']]
y_train_reg = train_data_reg['Distance']

# 重新訓練模型（fine tuning）
model_reg_finetuned = base_model_reg
model_reg_finetuned.fit(X_train_reg, y_train_reg)

# 儲存更新後的模型
joblib.dump(model_reg_finetuned,f'{root}/regressor_model_{ap}_best_{weekrepresent[1][0]}week_to_{weekrepresent[1][1]}week_{dataamount}dataPerRP.pkl')



['AP1 AP4 best (MDE_0.0093Accuracy_0.9919 )/regressor_model_AP1&AP4_best_0week_to_1week_20dataPerRP.pkl']

In [ ]:
### train data
train_data['AP2_Distance_predicted'] = np.nan
train_data['AP3_Distance_predicted'] = np.nan

# 利用 AP2_Rssi 預測 AP2_Distance_predicted
mask_ap2 = train_data['AP2_Rssi'].notna()
train_data.loc[mask_ap2, 'AP2_Distance_predicted'] = model_reg_finetuned.predict(
    train_data.loc[mask_ap2, ['AP2_Rssi']].rename(columns={'AP2_Rssi': 'Rssi'})
)

# 利用 AP3_Rssi 預測 AP3_Distance_predicted
mask_ap3 = train_data['AP3_Rssi'].notna()
train_data.loc[mask_ap3, 'AP3_Distance_predicted'] = model_reg_finetuned.predict(
    train_data.loc[mask_ap3, ['AP3_Rssi']].rename(columns={'AP3_Rssi': 'Rssi'})
)

### val data
val_data['AP2_Distance_predicted'] = np.nan
val_data['AP3_Distance_predicted'] = np.nan

# 利用 AP2_Rssi 預測 AP2_Distance_predicted
mask_ap2 = val_data['AP2_Rssi'].notna()
val_data.loc[mask_ap2, 'AP2_Distance_predicted'] = model_reg_finetuned.predict(
    val_data.loc[mask_ap2, ['AP2_Rssi']].rename(columns={'AP2_Rssi': 'Rssi'})
)

# 利用 AP3_Rssi 預測 AP3_Distance_predicted
mask_ap3 = val_data['AP3_Rssi'].notna()
val_data.loc[mask_ap3, 'AP3_Distance_predicted'] = model_reg_finetuned.predict(
    val_data.loc[mask_ap3, ['AP3_Rssi']].rename(columns={'AP3_Rssi': 'Rssi'})
)

### remaining data
remaining_data['AP2_Distance_predicted'] = np.nan
remaining_data['AP3_Distance_predicted'] = np.nan

# 利用 AP2_Rssi 預測 AP2_Distance_predicted
mask_ap2 = remaining_data['AP2_Rssi'].notna()
remaining_data.loc[mask_ap2, 'AP2_Distance_predicted'] = model_reg_finetuned.predict(
    remaining_data.loc[mask_ap2, ['AP2_Rssi']].rename(columns={'AP2_Rssi': 'Rssi'})
)

# 利用 AP3_Rssi 預測 AP3_Distance_predicted
mask_ap3 = remaining_data['AP3_Rssi'].notna()
remaining_data.loc[mask_ap3, 'AP3_Distance_predicted'] = model_reg_finetuned.predict(
    remaining_data.loc[mask_ap3, ['AP3_Rssi']].rename(columns={'AP3_Rssi': 'Rssi'})
)

# 更新 DNN 模型用的特徵欄位，將 regressor 預測值加入
selected_columns_dnn = selected_columns + ['AP2_Distance_predicted', 'AP3_Distance_predicted']


C:\Users\吳定洋\AppData\Local\Temp\ipykernel_1044\2456486366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['AP2_Distance_predicted'] = np.nan
C:\Users\吳定洋\AppData\Local\Temp\ipykernel_1044\2456486366.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['AP3_Distance_predicted'] = np.nan


['Label',
 'AP1_Distance (mm)',
 'AP4_Distance (mm)',
 'AP1_StdDev (mm)',
 'AP4_StdDev (mm)',
 'AP1_Rssi',
 'AP2_Rssi',
 'AP3_Rssi',
 'AP4_Rssi',
 'AP2_Distance_predicted',
 'AP3_Distance_predicted']

In [90]:

# **轉換為 NumPy 陣列**
X_train, y_train = train_data.drop(columns=['label']).values, train_data['label'].values
X_val, y_val = val_data.drop(columns=['label']).values, val_data['label'].values
X_test, y_test = remaining_data.drop(columns=['label']).values, remaining_data['label'].values


# **計算每個 Set 內各 Label 的資料數量**
train_label_counts = pd.Series(y_train).value_counts().sort_index()
val_label_counts = pd.Series(y_val).value_counts().sort_index()
test_label_counts = pd.Series(y_test).value_counts().sort_index()

# **確保所有 Labels 都有出現在三個 Set 裡**
all_labels = sorted(set(train_label_counts.index) | set(val_label_counts.index) | set(test_label_counts.index))
label_distribution = pd.DataFrame(index=all_labels)

label_distribution["Training Set"] = train_label_counts
label_distribution["Validation Set"] = val_label_counts
label_distribution["Test Set"] = test_label_counts

# **用 0 填補缺失值（表示該 Label 在該 Set 中沒有數據）**
label_distribution = label_distribution.fillna(0).astype(int)

# from IPython.display import display
# display(label_distribution)

X_train

array([[1948.        , 8799.        ,  -61.        , ...,  -64.        ,
        1077.        , 1076.        ],
       [3335.        , 8657.        ,  -58.        , ...,  -60.        ,
         394.        , 2299.        ],
       [4687.37102473, 8174.        ,  -60.95053004, ...,  -57.        ,
         828.96466431,  764.        ],
       ...,
       [1772.        , 8877.        ,  -61.        , ...,  -64.        ,
        1146.        ,  994.        ],
       [3452.        , 1914.        ,  -53.        , ...,  -52.        ,
         331.        ,  418.        ],
       [ 561.        , 3877.        ,  -54.        , ...,  -48.        ,
         340.        ,  481.        ]])

In [ ]:
for i in range(number_of_week):

# 載入 regressor
    base_model_reg = joblib.load(f'{root}/regressor_model_{ap}_best_{weekrepresent[i][0]}week_to_{weekrepresent[i][1]}week_{dataamount}dataPerRP.pkl')
# 載入 regressor dnn
    base_model_dnn = load_model(f'{root}/regressor_dnn_{ap}_best_{weekrepresent[i][0]}week_to_{weekrepresent[i][1]}week_{dataamount}dataPerRP.h5')
# 載入 scaler
    scaler = joblib.load(f'{root}/scaler_regressor_dnn_{ap}_best_0.pkl')

# 讀取測試資料 2024_12_21   2024_12_27   2025_01_03   2025_01_10   2025_02_28
    test_file_path = f"timestamp_allignment_Balanced_{alldatadate[i]}_rtt_logs.csv"  # 測試資料的檔案名稱
    date_test = f"{alldatadate[i]}"
    modelname = f"regressor_DNN {ap}s BEST_{dataamount}data_{alldatadate[i]}"
    data = pd.read_csv(test_file_path, usecols=selected_columns)

# DNN transfer learnging 凍結所有層
    for layer in base_model_dnn.layers[:-1]:  # 除了最後一層 (Output Layer)
        layer.trainable = False

    # 確認哪些層可訓練
    base_model_dnn.summary()

    base_model_dnn.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    
    processed_data = data.copy()

    # 資料填補：以每個 Label 群組內的平均值填補缺失值
    data_imputed = processed_data.groupby(label_column).apply(
        lambda group: group.fillna(group.mean())
    ).reset_index()
    data_imputed
    

    reverse_label_mapping = {v: int(k) - 1 for k, v in label_mapping.items()}  # 讓數字標籤 -1

    y = data_imputed[target_column]

    y_numeric = y.map(reverse_label_mapping)
    y_numeric

    # 把label部分拿掉
    X = data_imputed.drop(columns=['level_1','Label'])
    print(X.head())

    # 轉為 DataFrame 方便操作
    data = pd.DataFrame(X)
    data['label'] = y_numeric  # 加入 label 欄位

    train_data_full = data.groupby('label', group_keys=False).sample(n=N_train, replace=False, random_state=42)
    train_data_full

    if N_val > 0:
        sss = StratifiedShuffleSplit(n_splits=1, test_size=N_val / N_train) # , random_state=42
        train_index, val_index = next(sss.split(train_data_full.drop(columns=['label']), train_data_full['label']))
        train_data = train_data_full.iloc[train_index]
        val_data = train_data_full.iloc[val_index]
        
    else:
        val_data = pd.DataFrame(columns=data.columns)  # 若沒有 validation data，建立空 DataFrame\
        train_data = train_data_full

    remaining_data = data.drop(train_data_full.index)

    
    # **轉換為 NumPy 陣列**
    X_train, y_train = train_data.drop(columns=['label']).values, train_data['label'].values
    X_val, y_val = val_data.drop(columns=['label']).values, val_data['label'].values
    X_test, y_test = remaining_data.drop(columns=['label']).values, remaining_data['label'].values


   # **計算每個 Set 內各 Label 的資料數量**
    train_label_counts = pd.Series(y_train).value_counts().sort_index()
    val_label_counts = pd.Series(y_val).value_counts().sort_index()
    test_label_counts = pd.Series(y_test).value_counts().sort_index()

    # **確保所有 Labels 都有出現在三個 Set 裡**
    all_labels = sorted(set(train_label_counts.index) | set(val_label_counts.index) | set(test_label_counts.index))
    label_distribution = pd.DataFrame(index=all_labels)

    label_distribution["Training Set"] = train_label_counts
    label_distribution["Validation Set"] = val_label_counts
    label_distribution["Test Set"] = test_label_counts

    # **用 0 填補缺失值（表示該 Label 在該 Set 中沒有數據）**
    label_distribution = label_distribution.fillna(0).astype(int)

    from IPython.display import display
    display(label_distribution)



##################################################################################################

# 訓練 regressor 並儲存
    ap1_data = X_train[['AP1_Rssi', 'AP1_Distance (mm)']].dropna().rename(
        columns={'AP1_Rssi': 'Rssi', 'AP1_Distance (mm)': 'Distance'}
    )
    ap4_data = X_train[['AP4_Rssi', 'AP4_Distance (mm)']].dropna().rename(
        columns={'AP4_Rssi': 'Rssi', 'AP4_Distance (mm)': 'Distance'}
    )
    
    train_data_reg = pd.concat([ap1_data, ap4_data], ignore_index=True)

    
    X_train_reg = train_data_reg[['Rssi']]
    y_train_reg = train_data_reg['Distance']

    # 重新訓練模型（fine tuning）
    model_reg_finetuned = base_model_reg
    model_reg_finetuned.fit(X_train_reg, y_train_reg)

    # 儲存更新後的模型
    model_reg_finetuned = joblib.dump(f'{root}/regressor_model_{ap}_best_{weekrepresent[i+1][0]}week_to_{weekrepresent[i+1][1]}week_{dataamount}dataPerRP.pkl')



# 使用之前訓練時的標準化器 (scaler) 來標準化測試數據
    X_scaled = scaler.transform(X)

    import time
    # 記錄開始時間
    start_time = time.time()


    from tensorflow.keras.callbacks import EarlyStopping

    if N_val > 0:
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    else:
        early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)


    # 確保變數命名一致
    X_train_small = X_train  # 確保這裡用的變數和前面一致
    y_train_small = y_train

    # 設定 batch_size
    batch_size = min(32, max(8, len(X_train_small) // 2))  # 避免 batch size 過大
    # batch_size = 32
    print(batch_size)

    if N_val > 0:
        base_model.fit(X_train_small, y_train_small,validation_data=(X_val, y_val), epochs=10000, batch_size=batch_size, callbacks=[early_stop])
    else:
        base_model.fit(X_train_small, y_train_small, epochs=10000, batch_size=batch_size, callbacks=[early_stop])


    # 記錄結束時間
    end_time = time.time()


    # Needsave
    # 計算訓練時間（秒）
    training_time = end_time - start_time
    print(f"訓練時間：{training_time:.2f} 秒")
    ALL_trainingtime.append(training_time)

    # Needsave
    loss, accuracy = base_model.evaluate(X_test, y_test)
    print(f"Evaluation on 90% unused data - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
    ALL_loss.append(loss)
    ALL_accuracy.append(accuracy)

    # 預測測試資料
    y_test_pred_numeric = base_model.predict(X_test_scaled)
    y_pred_classes = np.argmax(y_test_pred_numeric, axis=1)

    # 轉換為原本的 Label
    y_test_pred_labels = [label_mapping[str(num + 1)] for num in y_pred_classes]  # 補回 +1
    y_test_pred_labels

    # 讀取測試資料的實際 Label
    y_test_actual = test_data_imputed[target_column]
    test_data_imputed

    # 取得預測與實際座標
    y_test_pred_coordinates = np.array([label_to_coordinates[label] for label in y_test_pred_labels])
    y_test_actual_coordinates = np.array([label_to_coordinates[label] for label in y_test_actual])

    # 計算 MDE (Mean Distance Error)
    distances = np.linalg.norm(y_test_pred_coordinates - y_test_actual_coordinates, axis=1)
    mean_mde = np.mean(distances)

    # 記錄每個 RP 的 MDE
    mde_report_test = {}
    for true_label, distance in zip(y_test_actual, distances):
        if true_label not in mde_report_test:
            mde_report_test[true_label] = []
        mde_report_test[true_label].append(distance)

    # 計算測試資料的 MDE 平均值
    mde_report_test_avg = {label: {"mde": np.mean(dists), "count": len(dists)} for label, dists in mde_report_test.items()}

    # 儲存 MDE 結果到 JSON 檔案
    test_file_path = f"{root}/{modelname}.json"
    with open(test_file_path, "w") as f:
        json.dump(mde_report_test_avg, f, indent=4)

    # Needsave
    print(f"Test Data MDE report saved to: {test_file_path}")
    print(f"\nTest Data Mean MDE: {mean_mde:.4f} meters")
    ALL_mean_mde.append(mean_mde)


    base_model.save(f'{root}/DNN_best_model{ap}_{weekrepresent[i+1][0]}week_to_{weekrepresent[i+1][1]}week_{dataamount}dataPerRP.h5')
